# MVD 11. cvičení


## 1. část - Collaborative Filtering (item-item)

Vytvořte si matici podle příkladu v přednášce na item-item přístup na snímku 24. Využijte centered cosine similarity pro výpočet podobností stejně jako v přednášce a vyberte dva nejbližší sousedy. Ověřte, že je výsledek v přednášce správný a implementujte funkci tak, aby bylo možné jednoduše spočítat i libovolné další vyhodnocení. 

In [1]:
import numpy as np
from colabFilter import ColabFilter

items =  np.asarray([
    [1,0,3,0,0,5,0,0,5,0,4,0],
    [0,0,5,4,0,0,4,0,0,2,1,3],
    [2,4,0,1,2,0,3,0,4,3,5,0],
    [0,2,4,0,5,0,0,4,0,0,2,0],
    [0,0,4,3,4,2,0,0,0,0,2,5],
    [1,0,3,0,3,0,0,2,0,0,4,0],
])

colabf = ColabFilter(items)
print("dist 0,0:",colabf.dist(0,0))
print("dist 0,1:",colabf.dist(0,1))
print("dist 0,2:",colabf.dist(0,2))
print("dist 0,3:",colabf.dist(0,3))
print("dist 0,4:",colabf.dist(0,4))
print("dist 0,5:",colabf.dist(0,5))
print("closest",colabf.closest())



dist 0,0: 0.9999999999999999
dist 0,1: -0.17854212213729673
dist 0,2: 0.41403933560541256
dist 0,3: -0.10245014273309601
dist 0,4: -0.30895719032666236
dist 0,5: 0.5870395085642741
closest (0.5870395085642741, (0, 5))


## Bonus - Content-based Filtering

Stáhněte si Kaggle dataset [Spotify Recommendation system](https://www.kaggle.com/bricevergnou/spotify-recommendation). Z datasetu budete potřebovat:

- data.csv = příznaky k jednotlivým skladbám + příznak liked pro klasifikaci

Úkolem je:

1. Načíst data z csv.
2. Vytvořit train (90 %) a test (10 %) split pomocí knihovny [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).
3. Vytvořit model logistické regrese (vlastní implementace / využití knihovny sklearn nebo jiné).
4. Vyhodnotit data na testovací sadě (např. metoda score u LogisticRegression).

**Skóre pro uznání úlohy musí být vyšší než 89 %.**

Dobrovolné: vytvořit graf predikovaných vs aktuálních hodnot

Získání bodu navíc: Využijte i další algoritmy pro binární klasifikaci.